In [1]:
import mlflow 
import warnings, git, hashlib, os
from prefect import flow, task 

In [2]:
#Github config

try:
    repo = git.Repo(search_parent_directories = True)
    sha = repo.head.commit.hexsha
except:
    warnings.warn('No github repository! Generating new SHA.')
    sha = hashlib.algorithms_guaranteed

In [3]:
#Mlflow config

name = 'Best Classifier'
experiment = mlflow.get_experiment_by_name(name)
if not experiment:
    experiment_id = mlflow.create_experiment(name)
experiment = mlflow.set_experiment(name)
run_params = {"experiment_id" : experiment.experiment_id,
              "description" : "Testing different classifiers for articial data with prefect and mlflow.",
              "tags" : {'release.version':'0.0.1'}}

In [4]:
print("Experiment ID: {}".format(experiment.experiment_id))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Tags: {}".format(experiment.tags))
print("Lifecycle stage: {}".format(experiment.lifecycle_stage))

Experiment ID: 1
Artifact Location: file:///home/lpfgarcia/Projects/MLOps/practical-prefect/mlruns/1
Tags: {}
Lifecycle stage: active


In [5]:
from sklearn.datasets import make_classification

@task
def generate_data(n_samples, n_features):
    return make_classification(n_samples, n_features, n_informative=2, n_redundant=2, random_state=42)

In [6]:
from sklearn.model_selection import train_test_split

@task
def split_data(X, y):
    return train_test_split(X, y, shuffle=False, test_size=int(len(X)*0.1))

In [7]:
@flow
def generate_split(n_samples, n_features):
    X, y = generate_data(n_samples, n_features)
    return split_data(X, y)

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

def classifiers():
    
    clf_list = [
        (KNeighborsClassifier(3), "Nearest Neighbors"),
        (SVC(kernel="linear", C=0.025), "Linear SVM"),
        (DecisionTreeClassifier(max_depth=5), "Decision Tree"),
        (RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1), "Random Forest"),
        (AdaBoostClassifier(), "AdaBoost"),
        (GaussianNB(), "Naive Bayes"),
    ]
    
    return clf_list

In [9]:
@task
def fit(clf, X_train, y_train):
    return clf.fit(X_train, y_train)

In [10]:
@task
def predict(model, X_test):
    return model.predict(X_test)

In [11]:
from sklearn.metrics import accuracy_score

@task
def performance(y_test, y_pred):
    return accuracy_score(y_test, y_pred)

In [12]:
@flow
def evaluate_classifier(clf, X_train, X_test, y_train, y_test):
    model = fit(clf, X_train, y_train)
    y_pred = predict(model, X_test)
    return model, y_pred, performance(y_test, y_pred)

In [13]:
@flow(version=sha)
def evaluate(samples, features):
    """Evaluating different classifier for each dataset configuration"""
    clf_list = classifiers()
    X_train, X_test, y_train, y_test = generate_split(samples, features)

    for clf, name in clf_list:
    
        with mlflow.start_run() as run:

            mlflow.log_param('name', name)
            mlflow.log_param('samples', samples)  
            mlflow.log_param('features', features)  

            model, y_pred, acc = evaluate_classifier(clf, X_train, X_test, y_train, y_test)

            mlflow.sklearn.log_model(
                sk_model = model,
                artifact_path = name
            )

            mlflow.log_metric('accuracy', acc)

In [14]:
samples = range(10000, 50000, 10000)
features = range(20, 30, 2)

combination = [(x,y) for x in samples for y in features]
    
for samples, features in combination:
    evaluate(samples, features)

14:00:27.598 | INFO    | prefect.engine - Created flow run 'mahogany-slug' for flow 'evaluate'
14:00:27.766 | INFO    | Flow run 'mahogany-slug' - Created subflow run 'green-sheep' for flow 'generate-split'
14:00:27.828 | INFO    | Flow run 'green-sheep' - Created task run 'generate_data-eeb4694a-0' for task 'generate_data'
14:00:27.829 | INFO    | Flow run 'green-sheep' - Executing 'generate_data-eeb4694a-0' immediately...
14:00:27.913 | INFO    | Task run 'generate_data-eeb4694a-0' - Finished in state Completed()
14:00:27.937 | INFO    | Flow run 'green-sheep' - Created task run 'split_data-b2f518fa-0' for task 'split_data'
14:00:27.938 | INFO    | Flow run 'green-sheep' - Executing 'split_data-b2f518fa-0' immediately...
14:00:28.003 | INFO    | Task run 'split_data-b2f518fa-0' - Finished in state Completed()
14:00:28.053 | INFO    | Flow run 'green-sheep' - Finished in state Completed()
14:00:28.152 | INFO    | Flow run 'mahogany-slug' - Created subflow run 'versed-unicorn' for flow

14:00:37.042 | INFO    | Flow run 'glistening-sambar' - Executing 'split_data-b2f518fa-0' immediately...
14:00:37.118 | INFO    | Task run 'split_data-b2f518fa-0' - Finished in state Completed()
14:00:37.171 | INFO    | Flow run 'glistening-sambar' - Finished in state Completed()
14:00:37.255 | INFO    | Flow run 'augmented-jellyfish' - Created subflow run 'quick-hornet' for flow 'evaluate-classifier'
14:00:37.310 | INFO    | Flow run 'quick-hornet' - Created task run 'fit-2157b1fd-0' for task 'fit'
14:00:37.310 | INFO    | Flow run 'quick-hornet' - Executing 'fit-2157b1fd-0' immediately...
14:00:37.372 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
14:00:37.398 | INFO    | Flow run 'quick-hornet' - Created task run 'predict-f915ca64-0' for task 'predict'
14:00:37.398 | INFO    | Flow run 'quick-hornet' - Executing 'predict-f915ca64-0' immediately...
14:00:37.555 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
14:00:37.574 | INFO    | Fl

14:00:46.260 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
14:00:46.283 | INFO    | Flow run 'zippy-crab' - Created task run 'predict-f915ca64-0' for task 'predict'
14:00:46.283 | INFO    | Flow run 'zippy-crab' - Executing 'predict-f915ca64-0' immediately...
14:00:46.422 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
14:00:46.442 | INFO    | Flow run 'zippy-crab' - Created task run 'performance-0da4127c-0' for task 'performance'
14:00:46.442 | INFO    | Flow run 'zippy-crab' - Executing 'performance-0da4127c-0' immediately...
14:00:46.488 | INFO    | Task run 'performance-0da4127c-0' - Finished in state Completed()
14:00:46.522 | INFO    | Flow run 'zippy-crab' - Finished in state Completed()
14:00:47.343 | INFO    | Flow run 'pink-okapi' - Created subflow run 'strict-tarantula' for flow 'evaluate-classifier'
14:00:47.397 | INFO    | Flow run 'strict-tarantula' - Created task run 'fit-2157b1fd-0' for task 'fit'
14:00:47.397 | INFO    

14:00:55.508 | INFO    | Flow run 'stirring-hoatzin' - Finished in state Completed()
14:00:56.320 | INFO    | Flow run 'classic-labradoodle' - Created subflow run 'vagabond-tody' for flow 'evaluate-classifier'
14:00:56.380 | INFO    | Flow run 'vagabond-tody' - Created task run 'fit-2157b1fd-0' for task 'fit'
14:00:56.380 | INFO    | Flow run 'vagabond-tody' - Executing 'fit-2157b1fd-0' immediately...
14:00:56.999 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
14:00:57.023 | INFO    | Flow run 'vagabond-tody' - Created task run 'predict-f915ca64-0' for task 'predict'
14:00:57.023 | INFO    | Flow run 'vagabond-tody' - Executing 'predict-f915ca64-0' immediately...
14:00:57.133 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
14:00:57.156 | INFO    | Flow run 'vagabond-tody' - Created task run 'performance-0da4127c-0' for task 'performance'
14:00:57.157 | INFO    | Flow run 'vagabond-tody' - Executing 'performance-0da4127c-0' immediately...

14:01:06.270 | INFO    | Flow run 'tuscan-mushroom' - Executing 'predict-f915ca64-0' immediately...
14:01:06.388 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
14:01:06.415 | INFO    | Flow run 'tuscan-mushroom' - Created task run 'performance-0da4127c-0' for task 'performance'
14:01:06.416 | INFO    | Flow run 'tuscan-mushroom' - Executing 'performance-0da4127c-0' immediately...
14:01:06.466 | INFO    | Task run 'performance-0da4127c-0' - Finished in state Completed()
14:01:06.505 | INFO    | Flow run 'tuscan-mushroom' - Finished in state Completed()
14:01:07.315 | INFO    | Flow run 'bizarre-sawfly' - Created subflow run 'pink-seal' for flow 'evaluate-classifier'
14:01:07.371 | INFO    | Flow run 'pink-seal' - Created task run 'fit-2157b1fd-0' for task 'fit'
14:01:07.372 | INFO    | Flow run 'pink-seal' - Executing 'fit-2157b1fd-0' immediately...
14:01:07.539 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
14:01:07.561 | INFO    | Flow

...

14:03:43.025 | INFO    | Flow run 'proficient-viper' - Created task run 'predict-f915ca64-0' for task 'predict'
14:03:43.025 | INFO    | Flow run 'proficient-viper' - Executing 'predict-f915ca64-0' immediately...
14:03:43.275 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
14:03:43.299 | INFO    | Flow run 'proficient-viper' - Created task run 'performance-0da4127c-0' for task 'performance'
14:03:43.299 | INFO    | Flow run 'proficient-viper' - Executing 'performance-0da4127c-0' immediately...
14:03:43.349 | INFO    | Task run 'performance-0da4127c-0' - Finished in state Completed()
14:03:43.428 | INFO    | Flow run 'proficient-viper' - Finished in state Completed()
14:03:44.239 | INFO    | Flow run 'warping-slug' - Created subflow run 'aloof-coua' for flow 'evaluate-classifier'
14:03:44.296 | INFO    | Flow run 'aloof-coua' - Created task run 'fit-2157b1fd-0' for task 'fit'
14:03:44.297 | INFO    | Flow run 'aloof-coua' - Executing 'fit-2157b1fd-0' immediatel

14:04:03.773 | INFO    | Flow run 'chestnut-squid' - Finished in state Completed()
14:04:04.597 | INFO    | Flow run 'enthusiastic-cobra' - Created subflow run 'sassy-tamarin' for flow 'evaluate-classifier'
14:04:04.654 | INFO    | Flow run 'sassy-tamarin' - Created task run 'fit-2157b1fd-0' for task 'fit'
14:04:04.655 | INFO    | Flow run 'sassy-tamarin' - Executing 'fit-2157b1fd-0' immediately...
14:04:12.737 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
14:04:12.757 | INFO    | Flow run 'sassy-tamarin' - Created task run 'predict-f915ca64-0' for task 'predict'
14:04:12.758 | INFO    | Flow run 'sassy-tamarin' - Executing 'predict-f915ca64-0' immediately...
14:04:13.314 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
14:04:13.337 | INFO    | Flow run 'sassy-tamarin' - Created task run 'performance-0da4127c-0' for task 'performance'
14:04:13.338 | INFO    | Flow run 'sassy-tamarin' - Executing 'performance-0da4127c-0' immediately...
14

14:04:34.716 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
14:04:34.739 | INFO    | Flow run 'secret-caracara' - Created task run 'performance-0da4127c-0' for task 'performance'
14:04:34.740 | INFO    | Flow run 'secret-caracara' - Executing 'performance-0da4127c-0' immediately...
14:04:34.787 | INFO    | Task run 'performance-0da4127c-0' - Finished in state Completed()
14:04:34.833 | INFO    | Flow run 'secret-caracara' - Finished in state Completed()
14:04:35.643 | INFO    | Flow run 'elated-jaguarundi' - Created subflow run 'puzzling-tuna' for flow 'evaluate-classifier'
14:04:35.699 | INFO    | Flow run 'puzzling-tuna' - Created task run 'fit-2157b1fd-0' for task 'fit'
14:04:35.699 | INFO    | Flow run 'puzzling-tuna' - Executing 'fit-2157b1fd-0' immediately...
14:04:36.146 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
14:04:36.167 | INFO    | Flow run 'puzzling-tuna' - Created task run 'predict-f915ca64-0' for task 'predict'
14:04

14:04:58.046 | INFO    | Flow run 'azure-flounder' - Created task run 'fit-2157b1fd-0' for task 'fit'
14:04:58.046 | INFO    | Flow run 'azure-flounder' - Executing 'fit-2157b1fd-0' immediately...
14:04:58.530 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
14:04:58.550 | INFO    | Flow run 'azure-flounder' - Created task run 'predict-f915ca64-0' for task 'predict'
14:04:58.550 | INFO    | Flow run 'azure-flounder' - Executing 'predict-f915ca64-0' immediately...
14:04:58.593 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
14:04:58.613 | INFO    | Flow run 'azure-flounder' - Created task run 'performance-0da4127c-0' for task 'performance'
14:04:58.613 | INFO    | Flow run 'azure-flounder' - Executing 'performance-0da4127c-0' immediately...
14:04:58.655 | INFO    | Task run 'performance-0da4127c-0' - Finished in state Completed()
14:04:58.683 | INFO    | Flow run 'azure-flounder' - Finished in state Completed()
14:04:59.467 | INFO    | Flow

14:05:22.391 | INFO    | Flow run 'splendid-raven' - Created task run 'performance-0da4127c-0' for task 'performance'
14:05:22.392 | INFO    | Flow run 'splendid-raven' - Executing 'performance-0da4127c-0' immediately...
14:05:22.441 | INFO    | Task run 'performance-0da4127c-0' - Finished in state Completed()
14:05:22.468 | INFO    | Flow run 'splendid-raven' - Finished in state Completed()
14:05:23.326 | INFO    | Flow run 'accurate-spider' - Created subflow run 'splendid-dormouse' for flow 'evaluate-classifier'
14:05:23.379 | INFO    | Flow run 'splendid-dormouse' - Created task run 'fit-2157b1fd-0' for task 'fit'
14:05:23.379 | INFO    | Flow run 'splendid-dormouse' - Executing 'fit-2157b1fd-0' immediately...
14:05:23.586 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
14:05:23.606 | INFO    | Flow run 'splendid-dormouse' - Created task run 'predict-f915ca64-0' for task 'predict'
14:05:23.606 | INFO    | Flow run 'splendid-dormouse' - Executing 'predict-f915ca